In [1]:
import os
from pathlib import Path
import re
import subprocess
from math import log10, floor
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
import plotly.graph_objects as go
from copy import deepcopy
import pandas as pd
import itertools
from pptx import Presentation
from pptx.util import Inches

from pymatgen.ext.matproj import MPRester, MPRestError
from monty.serialization import loadfn, dumpfn
from pymatgen import Structure, Composition
from pymatgen.analysis.reaction_calculator import ComputedEntry, ComputedReaction
from pymatgen.analysis.local_env import CrystalNN
from pymatgen.analysis.defects.utils import ChargeDensityAnalyzer
from pymatgen.util.plotting import pretty_plot, periodic_table_heatmap
from pymatgen.command_line.bader_caller import BaderAnalysis
from pymatgen.core import periodic_table
from pymatgen.core.sites import Site, PeriodicSite
from pymatgen.core.bonds import CovalentBond, bond_lengths
from pymatgen.io.vasp.outputs import Elfcar, VolumetricData, Chgcar

from scipy.stats import linregress, skew
from scipy.interpolate import RegularGridInterpolator
from sklearn.metrics import max_error, mean_absolute_error, mean_squared_error

In [2]:
PROJECT = 'SCAN project'

workdir = Path(re.sub("(?<={})[\w\W]*".format(PROJECT), "", str(Path.cwd())))
os.chdir(workdir)

data_dir = workdir / 'SCAN_elfcar_v1'
output_dir = workdir / 'vasp_files'

In [3]:
print(workdir)
print(data_dir)
print(output_dir)

/mnt/c/Users/Ayush/Desktop/elf-analysis
/mnt/c/Users/Ayush/Desktop/elf-analysis/SCAN_elfcar_v1
/mnt/c/Users/Ayush/Desktop/elf-analysis/vasp_files


In [26]:
compounds = []

In [29]:
def reorganize_files(): 
    print('Reorganizing for:')
    count, empty = 0, 0
    # loop through the directories containing the calcs
    block_dir = data_dir
    
    for d1 in block_dir.iterdir():
        if '.DS_Store' in str(d1):
            empty += 1
            continue
        
        # loop through the "launcher" dirs in each "block"
        for d2 in d1.iterdir():
            count+=1
            
            if '.DS_Store' in str(d2):
                empty += 1
                continue
                
            def file_helper(poscar_path, chgcar_path, aeccar0_path, aeccar2_path):
                formula = Structure.from_file(poscar_path).composition.reduced_formula
                compounds.append(formula)
                print('---------------------------------------------------------------')
                print(formula)
                
                if not os.path.isdir('vasp_files/by_compound/{}/'.format(formula)):
                    subprocess.run(['mkdir', 'vasp_files/by_compound/{}/'.format(formula)])
                    print('\tFolder for', formula, 'successfully made!')
                else:
                    print('\tFolder for', formula, 'already exists!')
                    
                target_dir = output_dir / 'by_compound' / formula
                
                if not os.path.isfile('vasp_files/by_compound/{}/CHGCAR.vasp'.format(formula)):           
                    chgcar = Chgcar.from_file(chgcar_path)
    #                 aeccar0 = Chgcar.from_file(aeccar0_path)
    #                 aeccar2 = Chgcar.from_file(aeccar2_path)

                    chgcar.write_file(Path(target_dir, 'CHGCAR.vasp'))
                    print('\tCHGCAR written')
    #                 aeccar0.write_file(Path(target_dir, 'AECCAR0.vasp'))
    #                 print('\tAECCAR0 written')
    #                 aeccar2.write_file(Path(target_dir, 'AECCAR2.vasp'))
    #                 print('\tAECCAR0 written')   
                else:
                    print('\tCHGCAR already exists')
                    
                
            try:
                poscar_path = d2 / 'POSCAR.relax2.gz'
                chgcar_path = d2 / 'CHGCAR.relax2.gz'
                aeccar0_path = d2 / 'AECCAR0.relax2.gz'
                aeccar2_path = d2 / 'AECCAR2.relax2.gz'    
                file_helper(poscar_path, chgcar_path, aeccar0_path, aeccar2_path)
            except FileNotFoundError as e:
                poscar_path = d2 / 'POSCAR.relax3.gz'
                chgcar_path = d2 / 'CHGCAR.relax3.gz'
                aeccar0_path = d2 / 'AECCAR0.relax3.gz'
                aeccar2_path = d2 / 'AECCAR2.relax3.gz'
                file_helper(poscar_path, chgcar_path, aeccar0_path, aeccar2_path)
            except TypeError as e:
                print('AECCAR, you naughty boy!')
            except Exception as exc:
                print({exc})
                continue
        
    print('---------------------------------------------------------------')    
    print('Done! Traversed {0} directories, of which {1} were empty'.format(count, empty))

In [28]:
#reorganize_files()
print(compounds)

Reorganizing for:
---------------------------------------------------------------
TiC
	Folder for TiC already exists!
	CHGCAR written
---------------------------------------------------------------
IrO2
	Folder for IrO2 already exists!
	CHGCAR written
---------------------------------------------------------------
TaN
	Folder for TaN already exists!
	CHGCAR written
---------------------------------------------------------------
TiO2
	Folder for TiO2 already exists!
	CHGCAR written
---------------------------------------------------------------
MgSO4
	Folder for MgSO4 already exists!
	CHGCAR written
---------------------------------------------------------------
MnBr2
	Folder for MnBr2 already exists!
	CHGCAR written
---------------------------------------------------------------
Mn3O4
	Folder for Mn3O4 already exists!
	CHGCAR written
---------------------------------------------------------------
Pt5Se4
	Folder for Pt5Se4 already exists!
	CHGCAR written
--------------------------------

In [31]:
compounds = ['TiC', 'IrO2', 'TaN', 'TiO2', 'MnBr2', 'Mn3O4', 'Pt5Se4', 'Al2S3', 'Fe2O3']

In [44]:
def run_bader():
    target_dir = output_dir / 'by_compound'
    for f in compounds:
        working_dir = target_dir / f
        os.chdir(working_dir)
        if not os.path.isfile('Bvol0001.dat'):
            subprocess.run(['bader', '-p', 'all_bader', 'CHGCAR.vasp'])
            print('Bader complete', f)
        else:
            print('Bader already ran', f)

In [42]:
run_bader()

Bader complete TiC
Bader complete IrO2
Bader complete TaN
Bader complete TiO2
Bader complete MnBr2
Bader complete Mn3O4
Bader complete Pt5Se4
Bader complete Al2S3
Bader complete Fe2O3


In [45]:
run_bader()

Bader already ran TiC
Bader already ran IrO2
Bader already ran TaN
Bader already ran TiO2
Bader already ran MnBr2
Bader already ran Mn3O4
Bader already ran Pt5Se4
Bader already ran Al2S3
Bader already ran Fe2O3


In [94]:
def retrieve_bader_results(return_dict={}):
    target_dir = output_dir / 'by_compound'
    for f in compounds:
        if not return_dict.get(f, None):
            chgcars = []
            working_dir = target_dir / f
            count = 0
            for entry in working_dir.iterdir():
                if re.search(r'Bvol', str(entry)):
                    chgcars.append(Chgcar.from_file(entry))
                    count += 1
            return_dict[f] = chgcars
            print(f, 'done:', count, 'files')
        else:
            print(f, 'already done')

In [95]:
chgcars_dict = {}

In [96]:
retrieve_bader_results(chgcars_dict)

TiC done: 55 files
IrO2 done: 6 files
TaN done: 13 files
TiO2 done: 6 files
MnBr2 done: 27 files
Mn3O4 done: 14 files
Pt5Se4 done: 42 files
Al2S3 done: 168 files
Fe2O3 done: 40 files


In [98]:
retrieve_bader_results(chgcars_dict)

TiC already done
IrO2 already done
TaN already done
TiO2 already done
MnBr2 already done
Mn3O4 already done
Pt5Se4 already done
Al2S3 already done
Fe2O3 already done


In [112]:
os.chdir(workdir)
chgcars_dict2 = {}
for f in chgcars_dict.keys():
    chgcars_dict2[f] = [chgcar.as_dict() for chgcar in chgcars_dict[f]]

In [ ]:
dumpfn(chgcars_dict2, 'lab_data/bader_chgcars.json')

In [105]:
def filter_zeros(chgcars):
    filtered = {}
    for f in chgcars.keys():
        lst, lst2 = chgcars[f], []
        for chg in lst:
            data = chg.data['total'].flatten()
            data = data[data != 0.0]
            lst2.append(data)
        filtered[f] = lst2
    return filtered

In [106]:
filtered_chgcars = filter_zeros(chgcars_dict)